In [29]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from webdriver_manager.chrome import ChromeDriverManager

# Set up Selenium with headless Chrome
options = Options()
options.add_argument("--headless")  
options.add_argument("--disable-gpu")
options.add_argument("--no-sandbox")

# Set up WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service, options=options)

# Stocks to scrape
stocks = ["ALK", "ALGT", "AAL", "DAL", "JBLU", "LUV", "UAL"]
all_data = []

for stock in stocks:
    url = f"https://www.quiverquant.com/congresstrading/stock/{stock}"
    driver.get(url)
    
    # Wait for table to load
    wait = WebDriverWait(driver, 10)
    wait.until(EC.presence_of_element_located((By.TAG_NAME, "table")))

    # Extract data from table
    table = driver.find_element(By.TAG_NAME, "table")
    rows = table.find_elements(By.TAG_NAME, "tr")
    
    headers = [header.text for header in rows[0].find_elements(By.TAG_NAME, "th")]
    for row in rows[1:]:  # Skip header row
        cols = row.find_elements(By.TAG_NAME, "td")
        all_data.append([stock] + [col.text for col in cols])  # Add stock symbol to each row

# Convert to DataFrame
df = pd.DataFrame(all_data, columns=["Stock"] + headers)

# Save to CSV
csv_filename = "/Users/saketh/Downloads/Daen Datasets/congress_trading_data.csv"
df.to_csv(csv_filename, index=False)

# Print the first few rows
print(df.head())

# Close the driver
driver.quit()

  Stock                                              Stock  \
0   ALK                    ALK\nALASKA AIR GROUP, INC.\nST   
1   ALK                    ALK\nALASKA AIR GROUP, INC.\nST   
2   ALK  ALK\nOF THE TRANSACTION, I SOLD THE STOCK. ALA...   
3   ALK                    ALK\nALASKA AIR GROUP, INC.\nST   
4   ALK                    ALK\nALASKA AIR GROUP, INC.\nST   

                  Transaction                  Politician          Filed  \
0      Sale\n$1,001 - $15,000  Earl Blumenauer\nHouse / D  Jan. 04, 2024   
1  Purchase\n$1,001 - $15,000  Earl Blumenauer\nHouse / D  Oct. 05, 2022   
2      Sale\n$1,001 - $15,000  Earl Blumenauer\nHouse / D  Apr. 01, 2022   
3  Purchase\n$1,001 - $15,000  Earl Blumenauer\nHouse / D  Apr. 01, 2022   
4   Sale\n$250,001 - $500,000  Victoria Spartz\nHouse / R  Jun. 20, 2021   

          Traded                                        Description  
0  Dec. 13, 2023          PART OF MY SPOUSE'S RETIREMENT PORTFOLIO.  
1  Sep. 30, 2022          PART